# Simple model

This is the code to train a simple model using keras. This seems to be unefficient, making a very complex and big CNN. However with no no more then 3500 images it converges well to almost 100% accuracy (on evaluation). Still only using, one model per person. Using only one type of attack.

The need of a more robust method would be good. A model that can generilize from multiple people and corectly discriminate live faces from spoofed ones, form different attacks. That would require more and better models of style tranfer.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras import callbacks
from keras.regularizers import l2
import pickle
import numpy as np
import datetime
import os
now_datetime  = datetime.datetime.now()
NAME = f"#Face_spoofing300x2_{now_datetime.day:02d}{now_datetime.month:02d}{now_datetime.year}_{now_datetime.hour:02d}{now_datetime.minute:02d}"
dir_pickle = "database_serialized"
dir_models_save = "models"
person = "001"


Using TensorFlow backend.


## Model format
This is a simple CNN, that is made of a convolutional and a dense layer. below you choose the numbers of filters of each concolutional layer, and the numbers of neurons of each dense layer

In [2]:
#format of the convolutional layer
format_convolution = [30,60,200]
#format of the dense layer
format_dense = [20,30,30]

## The model
The model creaeted here is enought for an almost ideal (acc = 98%) spoof detection model, as expected from a liveness detection methods that include deep learning as shown on many [studies](https://www.emerald.com/insight/content/doi/10.1108/SR-08-2015-0136/full/html) before.

However the artifical creation of spoofed images, may reduce its performance. For 2 different reasons:
1. Non perfect spoofed images. The creating of a model that can generate an spoofed image relayes much on the single image and the trainning data used to create such model. If both are not manage correctly, the creation of bad models may occur. Not to mention that some types of attacks are really hard to imitate e.g. print attacks.
2. Reduced numbers of data. There isn't much data online for this type of problem, and it to the creation of individuals models for each person may prove unresenable, for each person must have minutes (about two) of video of their faces, for a better conversion of the CNN.

For those reasons, it would be needed a deep leraning method that requires less data, to generalize. Something that could be accuired by [this](https://www.sciencedirect.com/science/article/pii/S1047320318301044) method.

In [3]:
def create_model(person, pickle_path):
    pickle_in = open(os.path.join(pickle_path,f"X{person}.pickle"),"rb")
    X = pickle.load(pickle_in)
    pickle_in.close()
    pickle_in = open(os.path.join(pickle_path,f"y{person}.pickle"),"rb")
    y = pickle.load(pickle_in)
    pickle_in.close()
    X = X/255.0
    model = Sequential()
    is_first = True
    # Convolutional layers
    for format_c in format_convolution:
        if is_first:
            model.add(layers.Conv2D(format_c,(3,3),input_shape=X.shape[1:]))
            is_first = False
        else:
            model.add(layers.Conv2D(format_c,(3,3)))
        model.add(layers.Activation("relu"))
        model.add(layers.MaxPool2D(pool_size=(3,3)))
    #Flatten the model if needed
    model.add(layers.Flatten())
    
    #Dense layers
    for format_d in format_dense:
        model.add(layers.Dropout(0.12))
        model.add(layers.Dense(format_d,activation="relu",kernel_regularizer=l2(0.002)))
    
    #Output layer
    model.add(layers.Dense(1,activation="sigmoid"))
    
    
    model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics= ["accuracy"])
    
    return X, y, model

In [4]:
# Create the model structure for the person
X, y, model = create_model(person,os.path.join(dir_pickle,person))

W1007 15:07:19.087997 15876 deprecation.py:506] From E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1007 15:07:19.401949 15876 deprecation.py:323] From E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Trainning your model
This line trains your model. It is recoomend to use tensorflow-gpu, and to be carefull if your gpu can handle the data, if  you are having *ResourceExhaustedError*, try reducing the **batch size**, or the sizes of your **tesors**.

Keep the eye on your **validation** because that shows your live performance without trainning bias. You may use checkpoints to get the best validation accuracy.

In [5]:
tensorboard = callbacks.TensorBoard(log_dir=os.path.join("tensorBoard_logs",NAME),write_graph=True,update_freq="batch",
                                    batch_size=5)
checkpoints = callbacks.ModelCheckpoint(filepath=os.path.join(dir_models_save,f"{person}{NAME}{format_convolution}{format_dense}.h5"),
                                        monitor="val_acc",
                                        mode = "max",
                                        verbose = 1,
                                        save_weights_only=False,
                                        save_best_only=True)

In [6]:
#Train model
model.fit(X,y,batch_size=25,validation_split=0.17,epochs=7,shuffle=False, callbacks=[checkpoints])

Train on 2942 samples, validate on 603 samples
Epoch 1/7
2925/2942 [============================>.] - ETA: 0s - loss: 0.3199 - acc: 0.9032
Epoch 00001: val_acc improved from -inf to 1.00000, saving model to models\001#Face_spoofing300x2_07102019_1507[30, 60, 200][20, 30, 30].h5
2942/2942 [==============================] - 33s 11ms/sample - loss: 0.3186 - acc: 0.9038 - val_loss: 0.0901 - val_acc: 1.0000
Epoch 2/7
2925/2942 [============================>.] - ETA: 0s - loss: 0.0869 - acc: 0.9973
Epoch 00002: val_acc did not improve from 1.00000
2942/2942 [==============================] - 42s 14ms/sample - loss: 0.0868 - acc: 0.9973 - val_loss: 0.0678 - val_acc: 1.0000
Epoch 3/7
2925/2942 [============================>.] - ETA: 0s - loss: 0.0795 - acc: 0.9887
Epoch 00003: val_acc did not improve from 1.00000
2942/2942 [==============================] - 43s 15ms/sample - loss: 0.0793 - acc: 0.9888 - val_loss: 0.0540 - val_acc: 1.0000
Epoch 4/7
2925/2942 [============================>.] - E

In [7]:
os._exit(00)